<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/%23Face%20Mask%20Dataset/Face_Mask_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [42]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

face-mask-12k-images-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [43]:
!unzip --qq face-mask-12k-images-dataset.zip

replace Face Mask Dataset/Test/WithMask/1163.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [46]:
import tensorflow as tf
from tensorflow.keras import layers,optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime

In [47]:
batch_size = 32
epoch = 10
lr = 0.001
width = height = 224

In [48]:
data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
)

train_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Train',
    target_size = (width,height),
    class_mode = 'binary',
    batch_size = batch_size,
    shuffle = True,
)

val_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Validation',
    target_size = (width,height),
    class_mode = 'binary',
    batch_size = batch_size,
    shuffle = True,
)

test_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Test',
    target_size = (width,height),
    class_mode = 'binary',
    batch_size = batch_size,
    shuffle = True,
)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [49]:
model = tf.keras.Sequential([
        layers.Conv2D(96, (11,11), activation='relu', input_shape=(width, height, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(256, (11,11), activation='relu', strides=(1, 1), padding='same'),
        layers.BatchNormalization(),
        
        layers.Conv2D(384, (3,3), activation='relu', strides=(1, 1), padding='same'),
        layers.BatchNormalization(),

        layers.Conv2D(256, (3,3), activation='relu', strides=(1, 1), padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2), strides=(2, 2)),


        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),

        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),

        layers.Dense(1, activation='softmax'),              
])

ResourceExhaustedError: ignored

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'],
              )

In [ ]:
model.fit(train_data,
          steps_per_epoch = train_data.samples/batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples/batch_size,
          epochs = epoch,
          )

Epoch 1/20
 26/156 [===>..........................] - ETA: 32s - loss: 4.5851 - accuracy: 0.8095

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(test_data)

16/16 [==============================] - 2s 108ms/step - loss: 198.7962 - accuracy: 0.4879


[198.79623413085938, 0.4879032373428345]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save('FaceMask.h5')
